# Qiskit device models

In order to explore what is feasible on IBM devices, you can now use tangelo to explore the types of errors one might expect when using particular IBM devices. These can be accessed by using the branch `qiskit-device-noise`.

The devices available are in `qiskit.test.mock`

In [1]:
# List all available Fake devices
from qiskit.providers.fake_provider import fake_provider
device_list = [func for func in dir(fake_provider) if callable(getattr(fake_provider, func))]
print(device_list)

['FakeAlmaden', 'FakeAlmadenV2', 'FakeArmonk', 'FakeArmonkV2', 'FakeAthens', 'FakeAthensV2', 'FakeBelem', 'FakeBelemV2', 'FakeBoeblingen', 'FakeBoeblingenV2', 'FakeBogota', 'FakeBogotaV2', 'FakeBrooklyn', 'FakeBrooklynV2', 'FakeBurlington', 'FakeBurlingtonV2', 'FakeCairo', 'FakeCairoV2', 'FakeCambridge', 'FakeCambridgeAlternativeBasis', 'FakeCambridgeV2', 'FakeCasablanca', 'FakeCasablancaV2', 'FakeEssex', 'FakeEssexV2', 'FakeGuadalupe', 'FakeGuadalupeV2', 'FakeHanoi', 'FakeHanoiV2', 'FakeJakarta', 'FakeJakartaV2', 'FakeJohannesburg', 'FakeJohannesburgV2', 'FakeKolkata', 'FakeKolkataV2', 'FakeLagos', 'FakeLagosV2', 'FakeLima', 'FakeLimaV2', 'FakeLondon', 'FakeLondonV2', 'FakeManhattan', 'FakeManhattanV2', 'FakeManila', 'FakeManilaV2', 'FakeMelbourne', 'FakeMelbourneV2', 'FakeMontreal', 'FakeMontrealV2', 'FakeMumbai', 'FakeMumbaiV2', 'FakeNairobi', 'FakeNairobiV2', 'FakeOpenPulse2Q', 'FakeOpenPulse3Q', 'FakeOurense', 'FakeOurenseV2', 'FakeParis', 'FakeParisV2', 'FakePoughkeepsie', 'FakeP

In [3]:
from tangelo.linq import Circuit, Gate, Simulator
from tangelo.linq.target import QiskitDevice

Below is an example for `"FakeMontreal"`. For any of the devices listed above, instantiate 
```python
sim_device = Simulator(target="qiskit_device", noise_model="Device_Name")
```

In [4]:
# Construct Quantum circuit
gates = [Gate('H', 0), Gate('CNOT', 0, 1), Gate('CNOT', 1, 2)]
circuit = Circuit(gates)

sim_ideal = Simulator('qiskit', n_shots=10000)
sim_montreal = Simulator(QiskitDevice, n_shots=10000, noise_model="FakeMontreal")

freqs_ideal, _ = sim_ideal.simulate(circuit)
freqs_montreal, _ = sim_montreal.simulate(circuit)

print(freqs_ideal)
print(freqs_montreal)


{'100': 0.4932, '000': 0.5068}
{'011': 0.0003, '001': 0.0067, '100': 0.465, '101': 0.0066, '111': 0.0003, '110': 0.0219, '010': 0.0226, '000': 0.4766}


## Choosing Qubits
Some qubits on the device have better properties. To test which qubits are best, we can assign `qubits_to_use`.

In [5]:
from tangelo.algorithms.variational import VQESolver
from tangelo.molecule_library import mol_H2_sto3g
from tangelo.algorithms.variational import BuiltInAnsatze

In [6]:
vqe_solver = VQESolver({"molecule": mol_H2_sto3g, "up_then_down": True, "ansatz": BuiltInAnsatze.HEA,
                        "qubit_mapping": "scbk", "ansatz_options": {"n_layers": 1, "rot_type": "real"}})
vqe_solver.build()
print(vqe_solver.simulate())

-1.1372696796984545


To use Montreal device qubits 0,1. We can obtain the expectation value as normal.

In [7]:
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit)

-1.0619500569075115

Looking at https://quantum-computing.ibm.com/services?services=systems, it appears that 19,20 are better qubits to use.

In [8]:
sim_montreal.qubits_to_use = [19,20]
sim_montreal = Simulator(target=QiskitDevice, n_shots=10000, noise_model='FakeMontreal', qubits_to_use=[19, 20])
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit)

-1.099147009676945

and indeed, you do get more accurate results using 19 and 20 compared to 0 and 1. 

Things to note. 
1. The qubits in the circuit must all be used and be in the `range(n_qubits)`. This can be done with the function `circuit.trim_qubits()`.
2. You can reorder as you like with `qubits_to_list=[25,24,16,19]`
3. The length of qubits_to_list must equal the number of qubits

## Choosing optimization level
Optimization level can be 0, 1, 2, 3. 3 being the most aggressive.

In [9]:
sim_montreal.opt_level = 3
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit)

# or equivalently reinitialize with
# sim_montreal = Simulator(target=QiskitDevice, n_shots=10000, noise_model=NoiseModel("FakeMontreal"), qubits_to_use=[19, 20], opt_level=3)

-1.0923270966480347

It does not seem to do much here

## measurement_error_mitigation
This will take extra time as it reruns the calibration circuit for each of the $2^{n_{qubits}}$ labels for 10,000 shots for each time. It does appear to work quite well here though.

In [10]:
sim_montreal.meas_mitt=True
freqs_montreal_mitt, _ = sim_montreal.simulate(vqe_solver.optimal_circuit)

print(freqs_montreal)
print(freqs_montreal_mitt)
print(freqs_ideal)

{'011': 0.0003, '001': 0.0067, '100': 0.465, '101': 0.0066, '111': 0.0003, '110': 0.0219, '010': 0.0226, '000': 0.4766}
{'00': 0.014592529290072351, '10': 0.0047959021696062235, '01': 0.002038250220629609, '11': 0.9785733183196919}
{'100': 0.4932, '000': 0.5068}


In [11]:
sim_montreal.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit)

-1.1274767449262875